# Web Scraping Reviews from Rotten Tomatoes

In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd 
import re

In [2]:
reviews=[] #for storing reviews
sentiment=[] #for storing the sentiment of reviews
date = [] #for storing dates

async def getData(season,page):
    res = requests.get(f'https://www.rottentomatoes.com/tv/killing_eve/s0{season}/reviews?type=&sort=&page={page}')
    soup = BeautifulSoup(res.text,'lxml')
    
    rev=soup.find_all('div',{'class':'critic__review-quote'})
    review = [i.text for i in rev]
    
    sentiments=soup.find_all('div',{'class':'review_icon'})
    find_sentiment= re.findall(r'rotten|fresh',str(sentiments))
    
    dates = soup.find_all('div',{'class':'critic__review-date subtle small'})
    d = [i.text for i in dates]
    
    reviews.extend(review)
    sentiment.extend(find_sentiment)
    date.extend(d)
    
    
for season in range(1,4):
    for page in range(1,6):
        await getData(season,page)

     

In [3]:
df = pd.DataFrame({'reviews': reviews,'sentiment': sentiment,'date':date})

In [4]:
df.head()

,reviews,sentiment,date
0,\n It manag...,fresh,"\n April 15, 20..."
1,\n Sandra O...,fresh,"\n January 11, ..."
2,\n What dis...,fresh,\n September 10...
3,\n [A] perf...,fresh,"\n May 14, 2019..."
4,\n The Brit...,fresh,"\n April 12, 20..."


In [5]:
df['sentiment'].replace(['fresh', 'rotten'], ['positive', 'negative'], inplace=True)

In [6]:
df = df.replace(r'\n',  '', regex=True)

In [7]:
df.head()

,reviews,sentiment,date
0,It manages...,positive,"April 15, 2020..."
1,Sandra Oh ...,positive,"January 11, 20..."
2,What disti...,positive,"September 10, ..."
3,[A] perfec...,positive,"May 14, 2019 ..."
4,The Britis...,positive,"April 12, 2019..."


In [8]:
df.to_csv('rotten_tomatoes_reviews.csv',index=False)